In [1]:
%%sql
CREATE TABLE users (
    id SERIAL PRIMARY KEY, 
    Username TEXT NOT NULL UNIQUE 
);


UsageError: Cell magic `%%sql` not found.


In [ ]:
CREATE TABLE projects (
    id SERIAL PRIMARY KEY,
    Name TEXT NOT NULL,
    user_id INTEGER REFERENCES users(id)
);

In [ ]:
CREATE TABLE experiments (
    id SERIAL PRIMARY KEY,
    Name TEXT NOT NULL,
    project_id INTEGER REFERENCES projects(id)
);

In [ ]:
CREATE TABLE documents (
    id SERIAL PRIMARY KEY,
    Name TEXT NOT NULL,
    experiment_id INTEGER REFERENCES experiments(id)
);

In [ ]:
CREATE TABLE probes (
    id SERIAL PRIMARY KEY,
    description TEXT NOT NULL,
    document_id INTEGER REFERENCES documents(id)
);

In [ ]:
CREATE TABLE samples (
    id SERIAL PRIMARY KEY,
    probe_id INTEGER REFERENCES probes(id)
    sample_count INTEGER,
    last_sample_time TIMESTAMPTZ,
    current_sampling_interval INTEGER,
    min_sampling_interval INTEGER,
    max_sampling_interval INTEGER
);

In [ ]:
CREATE TABLE spectra (
    id SERIAL PRIMARY KEY,
    sample_id INTEGER REFERENCES samples(id), 
    type TEXT CHECK (type IN ('raw', 'background', 'treated')),
    file_path TEXT NOT NULL,
    recorded_at TIMESTAMPTZ DEFAULT NOW()
);

In [ ]:
CREATE TABLE probe_temperature_trends (
    id SERIAL PRIMARY KEY,
    probe_id INTEGER REFERENCES probes(id),
    recorded_at TIMESTAMPTZ DEFAULT NOW(),
    temperature_celsius NUMERIC
);

In [ ]:
SELECT
    p.description,
    s.sample_count,
    s.last_sample_time,
    sr.file_path AS raw_spectra,
    sb.file_path AS background_spectra
FROM probes p
JOIN samples s ON p.id = s.probe_id
LEFT JOIN spectra sr ON sr.sample_id = s.id AND sr.type = 'raw'
LEFT JOIN spectra sb ON sb.sample_id = s.id AND sb.type = 'background'
WHERE p.id = 123;


In [ ]:
pip install graphviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from graphviz import Digraph

# Initialize the graph
dot = Digraph(comment='PostgreSQL Schema ERD', format='png')
dot.attr(rankdir='LR')

# Define tables and fields
tables = {
    "users": ["id", "username"],
    "projects": ["id", "name", "user_id (FK)"],
    "experiments": ["id", "name", "project_id (FK)"],
    "documents": ["id", "name", "experiment_id (FK)"],
    "probes": [
        "id", "description", "document_id (FK)",
        "latest_temperature_celsius", "latest_temperature_time"
    ],
    "samples": [
        "id", "probe_id (FK)", "sample_count", "last_sample_time",
        "current_sampling_interval", "min_sampling_interval", "max_sampling_interval"
    ],
    "spectra": ["id", "sample_id (FK)", "type", "file_path", "recorded_at"]
}

# Add nodes
for table, fields in tables.items():
    label = f"{table}|{'|'.join(fields)}"
    dot.node(table, shape='record', label="{" + label + "}")

# Foreign key relationships
relationships = [
    ("projects", "users"),
    ("experiments", "projects"),
    ("documents", "experiments"),
    ("probes", "documents"),
    ("samples", "probes"),
    ("spectra", "samples")
]

# Add edges
for src, dst in relationships:
    dot.edge(src, dst)

# Render to file
dot.render('schema_erd', format='png', view=True)


ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH